In [0]:
import tensorflow as tf

In [2]:
if tf.gfile.Exists('./fenwicks'):
  tf.gfile.DeleteRecursively('./fenwicks')
!git clone https://github.com/fenwickslab/fenwicks.git

Cloning into 'fenwicks'...
remote: Enumerating objects: 133, done.
remote: Counting objects: 100% (133/133), done.
remote: Compressing objects: 100% (77/77), done.
remote: Total 133 (delta 62), reused 99 (delta 35), pack-reused 0
Receiving objects: 100% (133/133), 33.20 KiB | 2.77 MiB/s, done.
Resolving deltas: 100% (62/62), done.


In [0]:
%load_ext autoreload
%autoreload 2

from fenwicks.datasets import *
from fenwicks.io import *
from fenwicks.vision.models.keras_models import *
from fenwicks.utils.colab_tpu import *
from fenwicks.tpu_estimator import *

In [0]:
BUCKET = 'gs://gs_colab' #@param {type:"string"}
PROJECT_NAME = 'dvc' #@param {type:"string"}
MODEL_NAME = 'InceptionResNetV2' #@param {type:"string"}

BATCH_SIZE = 128 #@param {type: "integer"}
EPOCHS = 20 #@param {type:"integer"}
IMG_SIZE = 299 #@param {type: "integer"}

LEARNING_RATE = 0.001 #@param {type:"number"}


In [5]:
setup_gcs()


For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
If you depend on functionality not listed there, please file an issue.



In [0]:
data_dir, work_dir, model_dir = get_gcs_dirs(BUCKET, PROJECT_NAME, MODEL_NAME)

In [0]:
ws_dir, ws_vars = InceptionResNetV2_weights(model_dir)

In [0]:
data_dir_local = untar_data(URLs.DVC, os.path.join('.', PROJECT_NAME))
train_dir_local = os.path.join(data_dir_local, 'dogscats/train')
valid_dir_local = os.path.join(data_dir_local, 'dogscats/valid')

In [0]:
train_fn = os.path.join(data_dir, 'train.tfrec')
valid_fn = os.path.join(data_dir, 'valid.tfrec')
_ = data_dir_tfrecord(train_dir_local, train_fn)
_ = data_dir_tfrecord(valid_dir_local, valid_fn)

In [0]:
class TransferLearningNet(tf.keras.Model):
  def __init__(self, c=256):
    super().__init__()
    self.ir2 = get_InceptionResNetV2()
    self.dense = tf.keras.layers.Dense(c, use_bias=False)
    self.bn = tf.keras.layers.BatchNormalization()
    self.linear = tf.keras.layers.Dense(2, use_bias=False)

  def call(self, x):
    return self.linear(tf.nn.relu(self.bn(self.dense(self.ir2(x)))))

In [0]:
total_steps = 23000 // BATCH_SIZE * EPOCHS

In [0]:
opt_func = adam_sgdr_one_cycle(LEARNING_RATE, total_steps)
model_fn = get_clf_model_func(TransferLearningNet, opt_func)

In [0]:
parser_train = get_tfexample_image_parser(IMG_SIZE, IMG_SIZE, augment=True)
parser_eval = get_tfexample_image_parser(IMG_SIZE, IMG_SIZE)

train_input_func = lambda params: tfrecord_ds(train_fn, parser_train,
  params['batch_size'], training=True)
valid_input_func = lambda params: tfrecord_ds(valid_fn, parser_eval, 
  params['batch_size'], training=False)

In [15]:
estimator = get_tpu_estimator(23000, 2000, model_fn, work_dir, 
                             ws_dir, ws_vars, BATCH_SIZE)

estimator.train(train_input_func, steps=total_steps)
estimator.evaluate(input_fn=valid_input_func, steps=1)

INFO:tensorflow:Using config: {'_model_dir': 'gs://gs_colab/work/dvc', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
cluster_def {
  job {
    name: "worker"
    tasks {
      key: 0
      value: "10.93.110.122:8470"
    }
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': None, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f4be1adf828>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': 'grpc://10.93.110.122:8470', '_evaluation_master': 'grpc://10.93.110.122:8470', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1, '_tpu_config': TPUConfig(iterations_per_loop=179, num_shards=None, num_cores_per_replic

{'accuracy': 0.989, 'global_step': 3580, 'loss': 0.09988603}

In [0]:
create_clean_dir(work_dir)